In [3]:
import numpy as np
import pandas as pd
import datetime

In [12]:
df = pd.read_csv("Nat_Gas.csv")
df["Dates"] = pd.to_datetime(df["Dates"])
df.head()

<ipython-input-12-9f179b6803bb>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Dates"] = pd.to_datetime(df["Dates"])


,Dates,Prices
0,2020-10-31,10.1
1,2020-11-30,10.3
2,2020-12-31,11.0
3,2021-01-31,10.9
4,2021-02-28,10.9


In [57]:
def priceContract(inj_date, withdraw_date, cost_price, sell_price, rate_injecion, rate_withdraw, max_vol, storage_cost, inj_withdraw_cost):


  total_cost_price = 0
  for i in range(len(cost_price)):
    total_cost_price += rate_injecion * cost_price[i]

  total_sell_price = 0
  for i in range(len(sell_price)):
    total_sell_price += rate_withdraw * sell_price[i]

  storage_duration = round((pd.to_datetime(withdraw_date[-1]) - pd.to_datetime(inj_date[0])).days/30.44)
  total_storage_cost = storage_duration * storage_cost

  total_inj_withdraw_cost =  inj_withdraw_cost * len(cost_price)

  val = (total_sell_price - total_cost_price) - total_storage_cost - total_inj_withdraw_cost

  # print("Cost Price : ", total_cost_price)
  # print("Sell Price : ", total_sell_price)
  # print("Storage Cost : ", total_storage_cost)
  return val

In [35]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def predict(date_to_predict):
  user_date = pd.to_datetime(date_to_predict)
  start_date = df["Dates"][0]

  diff_days = user_date - start_date
  diff_months = round(diff_days.days/30.44)
  p,d,q = 2,1,2
  P,D,Q = 1,0,1

  model = SARIMAX(df["Prices"], order=(p, q, d), seasonal_order=(P, Q, D, 12))
  fitted = model.fit()

  prediction = fitted.predict(diff_months)
  return prediction[diff_months]


In [61]:
import datetime

injection_date = ["2/10/2024","6/1/2025"]
withdraw_date = ["12/20/2024","4/10/2025"]

cost_price = [predict(x) for x in injection_date] # in $/MMBtu
sell_price = [predict(x) for x in withdraw_date]  # in $/MMBtu

storage_cost = 1e5 # 100k per month
rate_inj = 5e6 # Mmbtu/day
rate_withdraw  = 5e6 # Mmbtu/day
max_vol = 1e9
inj_withdraw_cost = 10000

contractValue = priceContract(injection_date, withdraw_date, cost_price, sell_price, rate_inj, rate_withdraw, max_vol, storage_cost,inj_withdraw_cost)
print("Value of contract ($) is : ", round(contractValue/1e6,2), " million dollars")


Value of contract ($) is :  3.62  million dollars
